In [261]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [262]:
df = pd.read_csv('../csv/Hotel_Reviews.csv')

In [263]:
df.isna().sum()

Hotel_Address                                    0
Additional_Number_of_Scoring                     0
Review_Date                                      0
Average_Score                                    0
Hotel_Name                                       0
Reviewer_Nationality                             0
Negative_Review                                  0
Review_Total_Negative_Word_Counts                0
Total_Number_of_Reviews                          0
Positive_Review                                  0
Review_Total_Positive_Word_Counts                0
Total_Number_of_Reviews_Reviewer_Has_Given       0
Reviewer_Score                                   0
Tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
dtype: int64

In [264]:
df.dropna(inplace=True)

In [265]:
df.shape

(512470, 17)

In [266]:
df['geom'] = df['lat'].map(str) + ',' + df['lng'].map(str)

In [267]:
unique_geom = df['geom'].unique()

In [268]:
unique_geom_df = pd.DataFrame(unique_geom)
unique_geom_df[0]

0           52.36057589999999,4.915968299999999
1        51.49188779999999,-0.19497060000000002
2               51.5137335,-0.10875119999999999
3       51.514218400000004,-0.18090320000000001
4                  48.874348100000006,2.2897334
                         ...                   
1467                      48.1935235,16.3669145
1468              48.2104468,16.371578099999997
1469            51.5020912,-0.15777950000000002
1470             51.499981,-0.19287910000000003
1471                      48.2037451,16.3356767
Name: 0, Length: 1472, dtype: object

In [269]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
def city(coord):
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    city = address.get('city', '')
    return city

def country(coord):
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    state = address.get('country', '')
    return state

In [ ]:
unique_geom_df['city'] = unique_geom_df[0].apply(city)

In [ ]:
unique_geom_df['country'] = unique_geom_df[0].apply(country)

In [ ]:
unique_geom_df['city'] = df[['city']]
unique_geom_df['country'] = df[['country']]

In [ ]:
unique_geom_df.set_index(0, inplace=True)

In [ ]:
df.set_index('geom', inplace=True)

In [ ]:
pd.merge(df, unique_geom_df, left_index=True, right_index=True, how='outer')

In [ ]:
df.isna().sum()

In [ ]:
df.to_csv('../csv/hotel_review_city_state.csv')